In [3]:
import requests
import os

# Define the range of value1 (GSM IDs) and value2 (IDs)
value1_start = 1570253
value1_end = 1570585
value2_start = 6420
value2_end = 6752

url_template = "https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?view=data&acc={value1}&id={value2}&db=GeoDb_blob122"
output_dir = "geo_downloads"
os.makedirs(output_dir, exist_ok=True)

v2 = value2_start - 1
for v1 in range(value1_start, value1_end + 1):
    gsm = f"GSM{v1}"
    v2 = v2 + 1
    url = url_template.format(value1=gsm, value2=v2)
    print(url)
    try:
        resp = requests.get(url, timeout=20)
        resp.raise_for_status()
        # Save to file
        filename = os.path.join(output_dir, f"{gsm}_{v2}.txt")
        with open(filename, "w", encoding="utf-8") as f:
            f.write(resp.text)
    except Exception as e:
        print(f"Failed to download {gsm}, {v2}: {e}")

print("Download complete.")

https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?view=data&acc=GSM1570253&id=6420&db=GeoDb_blob122
https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?view=data&acc=GSM1570254&id=6421&db=GeoDb_blob122
https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?view=data&acc=GSM1570255&id=6422&db=GeoDb_blob122
https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?view=data&acc=GSM1570256&id=6423&db=GeoDb_blob122
https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?view=data&acc=GSM1570257&id=6424&db=GeoDb_blob122
https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?view=data&acc=GSM1570258&id=6425&db=GeoDb_blob122
https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?view=data&acc=GSM1570259&id=6426&db=GeoDb_blob122
https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?view=data&acc=GSM1570260&id=6427&db=GeoDb_blob122
https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?view=data&acc=GSM1570261&id=6428&db=GeoDb_blob122
https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?view=data&acc=GSM1570262&id=6429&db=GeoDb_blob122
https://www.ncbi.nlm.nih.gov/g

In [7]:
import pandas as pd
import re
import os
from io import StringIO

def extract_idref_value(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()
    match = re.search(r'<pre>(.*?)</pre>', text, re.DOTALL | re.IGNORECASE)
    if not match:
        raise ValueError(f"No <pre> block found in {file_path}")
    pre_data = match.group(1)
    data_lines = [line for line in pre_data.splitlines() if line and not line.startswith('#')]
    for i, line in enumerate(data_lines):
        line_clean = re.sub(r'<.*?>', '', line).strip()
        if line_clean.startswith("ID_REF"):
            header_idx = i
            break
    else:
        raise ValueError(f"No header with ID_REF found in {file_path}")
    table_lines = data_lines[header_idx:]
    table_lines = [re.sub(r'<.*?>', '', l).strip() for l in table_lines]
    df = pd.read_csv(StringIO('\n'.join(table_lines)), sep='\t')
    df = df[['ID_REF', 'VALUE']]
    return df

# Gather all filenames in the folder
folder = "geo_downloads"
value_dfs = []
filenames = sorted([f for f in os.listdir(folder) if f.endswith(".txt")])  # Adjust box as needed

for f in filenames:
    abs_path = os.path.join(folder, f)
    sample_id = os.path.splitext(f)[0]  # Or extract GSM ID, as needed
    df = extract_idref_value(abs_path)
    df = df.rename(columns={"VALUE": sample_id})
    value_dfs.append(df)

# Merge all by 'ID_REF'
from functools import reduce
global_df = reduce(lambda left, right: pd.merge(left, right, on="ID_REF"), value_dfs)

global_df.to_csv("all_mice_GSE64398.csv", index=False)
print(global_df.head())

          ID_REF  GSM1570253_6420  GSM1570254_6421  GSM1570255_6422  \
0  0610005K03RIK          6.43230          6.49524          6.62031   
1  0610006F02RIK          6.48671          6.70625          6.81477   
2  0610006I08RIK         11.52560         11.49350         11.04880   
3  0610006K04RIK          8.89335          8.89025          8.93425   
4  0610007C21RIK         12.08650         11.91080         11.84890   

   GSM1570256_6423  GSM1570257_6424  GSM1570258_6425  GSM1570259_6426  \
0          6.63890          6.86916          6.51808          6.36677   
1          6.70933          6.70996          6.55293          6.69918   
2         11.33660         11.45820         12.15340         11.47650   
3          8.81467          9.04093          9.57344          8.96465   
4         11.97640         12.10390         12.49410         12.02650   

   GSM1570260_6427  GSM1570261_6428  ...  GSM1570576_6743  GSM1570577_6744  \
0          6.70498          6.89317  ...          6.5596

In [41]:
import re
import pandas as pd
from pathlib import Path

EXPECTED_HEADER = [
    "Group",
    "Accession",
    "Title",
    "Source name",
    "Sample section",
    "Hybridisation day",
    "Characteristics",
    "Grown with",
    "Genotype",
]

def parse_txt_to_df(path: str | Path) -> pd.DataFrame:
    path = Path(path)
    with path.open("r", encoding="utf-8") as f:
        # Strip whitespace-only lines
        lines = [ln.strip() for ln in f if ln.strip() != ""]

    # Find the header line (starts with "Group")
    try:
        header_idx = next(
            i for i, ln in enumerate(lines) if ln.split("\t")[0].strip() == "Group"
        )
    except StopIteration:
        raise ValueError('Header line starting with "Group" not found')

    # Process records after header
    i = header_idx + 1
    rows: list[list[str]] = []

    while i < len(lines):
        # Each record starts with a line that's exactly "-"
        if lines[i] != "-":
            i += 1
            continue

        group = lines[i]  # "-"
        i += 1
        if i >= len(lines):
            break

        # Next line: 6 tab-separated fields
        part1 = lines[i].split("\t")
        i += 1
        if i >= len(lines):
            break

        # Next line: 2 tab-separated fields
        part2 = lines[i].split("\t")
        i += 1

        if len(part1) != 6 or len(part2) != 2:
            raise ValueError(
                f"Unexpected column counts for a record near line {i}. "
                f"Got part1={len(part1)} fields, part2={len(part2)} fields."
            )

        row = [group] + part1 + part2  # total 1 + 6 + 2 = 9 columns
        rows.append(row)

    df = pd.DataFrame(rows, columns=EXPECTED_HEADER)

    # Extract integer age from "Characteristics" (e.g., "age(weeks): 32" -> 32)
    df["Age_weeks"] = (
        df["Characteristics"].str.extract(r"(\d+)")[0].astype("Int64")
    )

    # Drop the original text column since you only want the integer age
    df = df.drop(columns=["Characteristics"])

    # Optional: parse the date and set dtypes you might want
    # df["Hybridisation day"] = pd.to_datetime(df["Hybridisation day"], errors="coerce")

    return df
# Usage:

df = parse_txt_to_df("geo_downloads/metadata.txt")
print(df.head())
df.to_csv("all_mice_GSE64398_metadata.csv", index=False)

  Group   Accession Title Source name Sample section Hybridisation day  \
0     -  GSM1570253   478         HIP              A        2013-03-04   
1     -  GSM1570254   809         HIP              B        2013-03-04   
2     -  GSM1570255  1462         HIP              C        2013-03-04   
3     -  GSM1570256  1819         HIP              D        2013-03-04   
4     -  GSM1570257  2155         HIP              E        2013-03-04   

   Grown with    Genotype  Age_weeks  
0  HET_TASTPM  HET_TASTPM         32  
1   HO_TASTPM   HO_TASTPM         16  
2         TAU        WILD         32  
3         TAU         TAU         16  
4       TAS10        WILD         72  


HUMAN

In [21]:
import pandas as pd
from io import StringIO

def read_group_dash_table(path):
    """
    Parse a table where each data row is preceded by a line with '-'
    indicating the 'Group' column value, and the actual row values are
    tab-separated on the next line.

    Assumes the first non-empty line is the tab-separated header including 'Group' as the first column.
    """
    with open(path, 'r', encoding='utf-8') as f:
        raw = f.read()

    # Keep non-empty lines, preserve tabs
    lines = [ln.rstrip('\r\n') for ln in raw.splitlines() if ln.strip()]

    if not lines:
        return pd.DataFrame()

    header = lines[0]  # must include 'Group' as first column
    record_lines = []
    i = 1
    n = len(lines)

    while i < n:
        if lines[i].strip() == '-':
            # The next non-empty line is the data row (tab-separated, without the Group value)
            j = i + 1
            while j < n and not lines[j].strip():
                j += 1
            if j >= n:
                break
            data_line = lines[j]
            # Prepend the group token to form a complete row
            record_lines.append(f"-\t{data_line}")
            i = j + 1
        else:
            # Skip any stray line (should not happen if the file strictly alternates '-' and data)
            i += 1

    table_text = header + '\n' + '\n'.join(record_lines)
    df = pd.read_csv(StringIO(table_text), sep='\t', dtype=str)
    return df

# Example usage:
# Save your content to 'pfc_metadata.txt' and run:
if __name__ == "__main__":
    df = read_group_dash_table("human/metadata.txt")

    # Optional: extract numeric age in years from 'Age (Ch2)' like '67 yrs'
    if 'Age (Ch2)' in df.columns:
        df['Age (Ch2)'] = (
            df['Age (Ch2)'].astype(str).str.extract(r'(\d+)', expand=False).astype(int)
        )
        
    print(df.head())
    
    df.to_csv("all_human_GSE33000_metadata.csv", index=False)

  Group   Accession  Title    Source name 1 Source name 2  \
0     -  GSM1423780  PFC_1  HBTRC_PF_Pool_1         PFC_1   
1     -  GSM1423781  PFC_2  HBTRC_PF_Pool_2         PFC_2   
2     -  GSM1423782  PFC_3  HBTRC_PF_Pool_3         PFC_3   
3     -  GSM1423783  PFC_4  HBTRC_PF_Pool_4         PFC_4   
4     -  GSM1423784  PFC_5  HBTRC_PF_Pool_5         PFC_5   

              Tissue (Ch1) Sample type (Ch1)             Tissue (Ch2)  \
0  prefrontal cortex brain         reference  prefrontal cortex brain   
1  prefrontal cortex brain         reference  prefrontal cortex brain   
2  prefrontal cortex brain         reference  prefrontal cortex brain   
3  prefrontal cortex brain         reference  prefrontal cortex brain   
4  prefrontal cortex brain         reference  prefrontal cortex brain   

   Age (Ch2) Gender (Ch2) Disease status (Ch2)  
0         67       female  Alzheimer's disease  
1         88         male  Alzheimer's disease  
2         62         male  Alzheimer's disease 

In [25]:
import requests
import os

# Define the range of value1 (GSM IDs) and value2 (IDs)
value1_start = 1423780
value1_end = 1424246
value2_start = 19975
value2_end = 20441

url_template = "https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?view=data&acc={value1}&id={value2}&db=GeoDb_blob113"
output_dir = "human"
os.makedirs(output_dir, exist_ok=True)

v2 = value2_start - 1
for v1 in range(value1_start, value1_end + 1):
    gsm = f"GSM{v1}"
    v2 = v2 + 1
    url = url_template.format(value1=gsm, value2=v2)
    print(url)
    try:
        resp = requests.get(url, timeout=20)
        resp.raise_for_status()
        # Save to file
        filename = os.path.join(output_dir, f"{gsm}_{v2}.txt")
        with open(filename, "w", encoding="utf-8") as f:
            f.write(resp.text)
        exit(1)
    except Exception as e:
        print(f"Failed to download {gsm}, {v2}: {e}")

print("Download complete.")

https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?view=data&acc=GSM1423780&id=19975&db=GeoDb_blob113
https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?view=data&acc=GSM1423781&id=19976&db=GeoDb_blob113
https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?view=data&acc=GSM1423782&id=19977&db=GeoDb_blob113
https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?view=data&acc=GSM1423783&id=19978&db=GeoDb_blob113
https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?view=data&acc=GSM1423784&id=19979&db=GeoDb_blob113
https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?view=data&acc=GSM1423785&id=19980&db=GeoDb_blob113
https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?view=data&acc=GSM1423786&id=19981&db=GeoDb_blob113
https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?view=data&acc=GSM1423787&id=19982&db=GeoDb_blob113
https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?view=data&acc=GSM1423788&id=19983&db=GeoDb_blob113
https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?view=data&acc=GSM1423789&id=19984&db=GeoDb_blob113
https://www.ncbi.nlm

In [27]:
import os
import pandas as pd
import re
from io import StringIO

def extract_idref_value_from_geo_html(file_path):
    """Extracts ID_REF and VALUE columns from GEO-style HTML file."""
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()

    # Extract <pre>...</pre>
    match = re.search(r'<pre>(.*?)</pre>', text, re.DOTALL|re.IGNORECASE)
    if not match:
        raise ValueError(f"No <pre> block found in {file_path}")
    pre_data = match.group(1)

    # Remove comment and empty lines
    lines = [line for line in pre_data.splitlines() if line and not line.startswith('#')]
    
    # Find header index
    for i, line in enumerate(lines):
        line_clean = re.sub(r'<.*?>', '', line).strip()
        if line_clean.startswith("ID_REF"):
            header_idx = i
            break
    else:
        raise ValueError(f'No "ID_REF" header in {file_path}')
    
    # Keep only actual data table, strip HTML tags
    table_lines = lines[header_idx:]
    table_lines = [re.sub(r'<.*?>', '', l).strip() for l in table_lines]

    # Read into DataFrame
    df = pd.read_csv(StringIO('\n'.join(table_lines)), sep='\t')
    if 'ID_REF' not in df or 'VALUE' not in df:
        raise ValueError(f"Validation error for columns in {file_path}")
    return df[['ID_REF', 'VALUE']]

# Directory containing your files
folder = 'human'
dfs = []
names = []

for fname in sorted(os.listdir(folder)):
    fpath = os.path.join(folder, fname)
    if not os.path.isfile(fpath) or fpath == 'metadata.txt':
        continue
    try:
        df = extract_idref_value_from_geo_html(fpath)
        name = os.path.splitext(fname)[0]
        df = df.rename(columns={'VALUE': name})
        dfs.append(df)
        names.append(name)
    except Exception as e:
        print(f"Skipping {fname}: {e}")

if not dfs:
    raise RuntimeError('No valid files found.')

# Merge on ID_REF
from functools import reduce
merged_df = reduce(lambda left, right: pd.merge(left, right, on="ID_REF"), dfs)

print(merged_df.head())
# Save if wanted
# merged_df.to_csv("all_human_GSE33000.tsv", sep='\t', index=False)

Skipping metadata.txt: No <pre> block found in human\metadata.txt
        ID_REF  GSM1423780_19975  GSM1423781_19976  GSM1423782_19977  \
0  10019475365         -0.016184          0.027599         -0.089661   
1  10019481149         -0.017105         -0.023574          0.044863   
2  10019495284          0.077440         -0.058746         -0.120759   
3  10019687586          0.033595          0.032914         -0.030928   
4  10019713746         -0.001819          0.032487         -0.015073   

   GSM1423783_19978  GSM1423784_19979  GSM1423785_19980  GSM1423786_19981  \
0         -0.020493         -0.051699          0.075485         -0.046606   
1          0.003438         -0.064108         -0.053226         -0.071823   
2          0.072692         -0.051002         -0.183919         -0.074997   
3         -0.033051         -0.049156         -0.011204          0.072451   
4         -0.010303          0.063070          0.113860         -0.055590   

   GSM1423787_19982  GSM1423788_19983 

In [43]:
import pandas as pd

# df = pd.read_csv("all_human_GSE33000.csv")  # or sep=","
df = pd.read_csv("all_mice_GSE64398.csv")  # or sep=","
new_cols = [df.columns[0]] + [c.split("_")[0] for c in df.columns[1:]]
df.columns = new_cols

# df.to_csv("all_human_GSE33000.csv", index=False)
df.to_csv("all_mice_GSE64398.csv", index=False)

In [47]:
import pandas as pd

# --- Read the two CSVs ---
# Replace these with your actual file paths
meta_path = "all_mice_GSE64398_metadata.csv"
expr_path = "all_mice_GSE64398.csv"

meta_df = pd.read_csv(meta_path)   # or sep="," depending on your file
expr_df = pd.read_csv(expr_path)   # adjust separator accordingly

# --- Long format: one row per (Sample, Gene) with metadata merged ---

# Melt expression data to long form
long_expr = expr_df.melt(id_vars="ID_REF", var_name="Accession", value_name="Expression")

# Merge metadata
combined_long = long_expr.merge(meta_df, on="Accession", how="left")

# Now combined_long has columns:
# ID_REF (gene), Accession (GSM ID), Expression (value), plus all metadata columns.

# Example: get all rows for GSM1570253
sample_id = "GSM1570253"
sample_long_df = combined_long[combined_long["Accession"] == sample_id]

# print("Long format for sample", sample_id)
# print(sample_long_df.head())

# --- Wide format per sample: single metadata row with all gene expressions as columns ---

# Pivot expression so genes become columns
wide_expr = expr_df.set_index("ID_REF").T  # rows = GSM IDs, columns = genes

# wide_expr index currently are GSM IDs (Accession); ensure name matches for merging
wide_expr.index.name = "Accession"

# Merge with metadata
combined_wide = meta_df.merge(wide_expr, on="Accession", how="left")

# Example: single wide row for GSM1570253
sample_wide_df = combined_wide[combined_wide["Accession"] == sample_id]

# print("Wide format for sample", sample_id)
# print(sample_wide_df.iloc[0, :15])  # show first few columns (metadata + some genes)

# --- Build a dictionary of per-sample DataFrames (long format) ---
sample_long_dfs = {
    acc: df for acc, df in combined_long.groupby("Accession")
}

# Access:
sample_GSM1570253_df = sample_long_dfs["GSM1570253"]

# --- Or dictionary for wide format (each is a single-row DataFrame) ---
sample_wide_dfs = {
    acc: combined_wide[combined_wide["Accession"] == acc] for acc in combined_wide["Accession"]
}

# --- Optional: function helpers ---

def get_sample_long(accession: str) -> pd.DataFrame:
    return combined_long[combined_long["Accession"] == accession].copy()

def get_sample_wide(accession: str) -> pd.DataFrame:
    return combined_wide[combined_wide["Accession"] == accession].copy()

# Example usage:
df_long = get_sample_long("GSM1570253")
print(df_long)
# df_wide = get_sample_wide("GSM1570253")

              ID_REF   Accession  Expression Group Title Source name  \
0      0610005K03RIK  GSM1570253     6.43230     -   478         HIP   
1      0610006F02RIK  GSM1570253     6.48671     -   478         HIP   
2      0610006I08RIK  GSM1570253    11.52560     -   478         HIP   
3      0610006K04RIK  GSM1570253     8.89335     -   478         HIP   
4      0610007C21RIK  GSM1570253    12.08650     -   478         HIP   
...              ...         ...         ...   ...   ...         ...   
12553           ZW10  GSM1570253     8.36117     -   478         HIP   
12554           ZXDA  GSM1570253    10.91640     -   478         HIP   
12555         ZYG11B  GSM1570253     6.93133     -   478         HIP   
12556            ZYX  GSM1570253    12.18030     -   478         HIP   
12557           ZZZ3  GSM1570253     6.55961     -   478         HIP   

      Sample section Hybridisation day  Grown with    Genotype  Age_weeks  
0                  A        2013-03-04  HET_TASTPM  HET_TAS